In [ ]:
# Install necessary libraries
!pip install --upgrade gdown
!pip install torch torchvision
!pip install diffusers==0.27.1
!pip install accelerate==0.28.0
!pip install transformers==4.35.2
!pip install google-colab


In [ ]:
# Import necessary libraries
import os
from google.colab import drive
from diffusers import StableDiffusionPipeline, ControlNetModel
from transformers import CLIPProcessor, CLIPModel
import torch


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')


In [ ]:
# Define paths for dataset, model, and output
drive_root = '/content/drive/MyDrive'
project_name = 'your_project_name_here'  # Change to your specific project name
dataset_dir = os.path.join(drive_root, 'SD Images/1024jpg')
output_dir = os.path.join(drive_root, 'SD Images/output')
model_save_dir = os.path.join(drive_root, 'SD Images/Models')

# Ensure directories exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(model_save_dir, exist_ok=True)

# Check contents of dataset directory
print('Dataset files:', os.listdir(dataset_dir))


In [ ]:
# Load the Stable Diffusion pipeline and the ControlNet model
sd_model_id = 'pt-sk/stable-diffusion-1.5'  # Model ID on Hugging Face
controlnet_id = 'controlnet_model_id_here'  # Placeholder for ControlNet model ID

# Load models from Hugging Face
pipe = StableDiffusionPipeline.from_pretrained(sd_model_id, torch_dtype=torch.float16).to('cuda')
controlnet = ControlNetModel.from_pretrained(controlnet_id, torch_dtype=torch.float16).to('cuda')

# Apply ControlNet to the pipeline
pipe.controlnet = controlnet


In [ ]:
# Define image processing parameters
def process_image(input_image_path, output_image_path):
    # Load and preprocess the image
    image = pipe.load_image(input_image_path)
    # Run the pipeline with the input image
    result = pipe(image=image)
    # Save the output
    result.images[0].save(output_image_path)

# Test process on a sample image
sample_image = os.path.join(dataset_dir, 'sample_image_01.jpg')
output_image = os.path.join(output_dir, 'output_image_01.jpg')
process_image(sample_image, output_image)


In [ ]:
# Load LoRA model from Google Drive
lora_model_path = os.path.join(model_save_dir, 'bootlicker.safetensors')
pipe.load_lora_weights(lora_model_path)

# Check that LoRA is applied
print('LoRA model loaded successfully')


In [ ]:
# Process a batch of images
for i in range(1, 50):
    input_image = os.path.join(dataset_dir, f'sample_image_{i:02d}.jpg')
    output_image = os.path.join(output_dir, f'output_image_{i:02d}.jpg')
    process_image(input_image, output_image)
    print(f'Processed image {i:02d}')


In [ ]:
# Function to save the model back to Google Drive
def save_model(model, output_dir):
    model.save_pretrained(output_dir)
    print(f'Model saved at {output_dir}')

# Save the updated pipeline with LoRA
save_model(pipe, model_save_dir)
